In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [15]:
#파일 삭제
import os
import shutil

def delete_files_in_folder(folder_path):
    # 폴더 안의 파일들을 리스트로 가져옵니다.
    file_list = os.listdir(folder_path)

    # 리스트에 있는 파일들을 하나씩 삭제합니다.
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

# 폴더 경로 설정
target_folder = "/content/drive/MyDrive/google_da"  # 여기에 삭제하고자 하는 폴더의 경로를 입력하세요.

# 폴더 안의 파일들을 삭제합니다.
delete_files_in_folder(target_folder)

FileNotFoundError: ignored

In [16]:
#day 파일 개수 세기

import os

def count_files_in_folder(folder_path):
    file_count = 0

    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

if __name__ == '__main__':
    folder_path = '/content/drive/MyDrive/night_1'  # 파일 개수를 세고자 하는 폴더 경로를 입력하세요.

    num_files = count_files_in_folder(folder_path)
    print(f'폴더 "{folder_path}" 내에 있는 파일 개수: {num_files}')

폴더 "/content/drive/MyDrive/night_1" 내에 있는 파일 개수: 908


In [17]:
#시작

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import numpy as np
import torch
class BaseAugmentation:
    def __init__(self, mean=[0.5, 0.5, 0.5], std=[0.5,0.5,0.5]):
        self.transform = A.Compose([
                A.Normalize(mean=mean, std=std, always_apply=True),
                ToTensorV2()
        ])
#A.augmentations.crops.transforms.RandomCrop(128, 128, always_apply=False,p=1.0)
def denormalize_image(image, mean=[0.5, 0.5, 0.5], std=[0.5,0.5,0.5]):

    mean = torch.tensor(np.array(mean).reshape(1,-1,1,1))
    std = torch.tensor(np.array(std).reshape(1,-1,1,1))

    # if len(image.shape) == 4 and image.shape[0]==16:
    #     image = image.squeeze()

    denorm_image = (image*std+mean).clamp_(0,1)

    return denorm_image

In [18]:
from torch import nn

class AdversarialLoss(nn.Module):
    def __init__(self):
        super(AdversarialLoss, self).__init__()
        self.loss = nn.BCEWithLogitsLoss()

    # generator
    def forward_G(self, d_g_x, real):
        return self.loss(d_g_x, real)

    # discriminator
    def forward_D(self, d_y ,real, d_g_x, fake):

        d_real_loss = self.loss(d_y, real)
        d_fake_loss = self.loss(d_g_x, fake)
        d_loss = (d_real_loss + d_fake_loss)/2
        return d_loss


class CycleConsistencyLoss(nn.Module):
    def __init__(self):
        super(CycleConsistencyLoss, self).__init__()
        self.loss_forward = nn.L1Loss()
        self.loss_backward = nn.L1Loss()

    def forward(self, x, y, f_g_x, g_f_y):

        loss_cyc = self.loss_forward(f_g_x, x) + self.loss_backward(g_f_y, y)

        return loss_cyc

class IdentityLoss(nn.Module):
    def __init__(self):
        super(IdentityLoss, self).__init__()
        self.loss_x = nn.L1Loss()
        self.loss_y = nn.L1Loss()

    def forward(self, x, y, f_y, g_x):
        loss_idt = self.loss_x(f_y, x) + self.loss_y(g_x, y)

        return loss_idt

In [19]:
import os
import glob
from torch.utils.data import Dataset, DataLoader
import torch
import cv2
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
from typing import List, Dict


class UnalignedDataset(Dataset):

    def __init__(self, data_root_A: str, data_root_B: str, is_train:bool=True, transforms=ToTensorV2(), num_workers: int=2):

        super(UnalignedDataset, self).__init__()
        self.data_root_A = data_root_A
        self.data_root_B = data_root_B
        self.is_train = is_train
        self.transforms = transforms

        paths_A = sorted(self._load_image_path(self.data_root_A))
        paths_B = sorted(self._load_image_path(self.data_root_B))

        self.image_paths_A, self.image_paths_B = self._adjust_dataset_length(paths_A, paths_B)


    def __len__(self):

        return len(self.image_paths_A)

    def __getitem__(self, index:int)->Dict:

        A = cv2.cvtColor(cv2.imread(self.image_paths_A[index]), cv2.COLOR_BGR2RGB)
        B = cv2.cvtColor(cv2.imread(self.image_paths_B[index]), cv2.COLOR_BGR2RGB)

        if self.transforms:
            A = self.transforms(image=A)['image']
            B = self.transforms(image=B)['image']

        return {'A': A, 'B': B}

    def _load_image_path(self, data_dir:str)->List[str]:

        image_path  = glob.glob(data_dir+"/*")

        return image_path

    def _adjust_dataset_length(self, paths_A:str, paths_B:str):
        min_len = min(len(paths_A), len(paths_B))
        return paths_A[:min_len], paths_B[:min_len]

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from typing import Union
from collections import OrderedDict
#Generator
class ResidualBlock(nn.Module):
    def __init__(self, in_channels:int, out_channels:int):

        super(ResidualBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.block = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, padding='same', padding_mode='reflect'),
            nn.InstanceNorm2d(self.out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, padding='same', padding_mode='reflect'),
            nn.InstanceNorm2d(self.out_channels)
        )

    def forward(self, x):
        output = self.block(x) + x # skip-connection

        return output


class Generator(nn.Module):
    def __init__(self, init_channel:int, kernel_size:int, stride:int, n_blocks:int=6):

        super(Generator, self).__init__()

        self.init_channel=init_channel
        self.kernel_size=kernel_size
        self.stride=stride
        self.n_blocks=n_blocks

        layers = OrderedDict()
        layers['conv_first'] = self._make_block(in_channels=3, out_channels=self.init_channel, kernel_size=7, stride=1, padding='same') # first layer

        # downsampling path (d_k) -> two downsampling blocks
        for i in range(2):
            ic = self.init_channel*(i+1)
            k = 2*ic
            layers[f'd_{k}'] = self._make_block(in_channels=ic, out_channels=k, kernel_size=self.kernel_size, stride=self.stride)

        # residual block (R_k) -> 6 or 9 blocks
        for i in range(self.n_blocks):
            layers[f'R{k}_{i+1}'] = ResidualBlock(k, k) # in_channel = out_channel로 동일한 channel dimension 유지

        # upsampling path (u_k) -> two upsampling blocks
        for i in range(2):
            k = int(k/2)
            layers[f'u_{k}'] = self._make_block(in_channels=k*2, out_channels=k, kernel_size=self.kernel_size, stride=self.stride, mode='u')

        # last conv layer
        layers['conv_last'] = nn.Conv2d(in_channels=self.init_channel, out_channels=3, kernel_size=7, stride=1, padding='same', padding_mode='reflect') # last conv layer (to rgb)
        layers['tanh'] = nn.Tanh()

        self.model = nn.Sequential(
            layers
        )

    def forward(self, x):
        op = self.model(x)
        assert op.shape == x.shape, f"output shape ({op.shape}) must be same with the input size ({x.shape})"
        return op

    def _make_block(self, in_channels:int, out_channels:int, kernel_size:int, stride:int, padding:Union[int,str]=1, mode:str='d'):


        block = []
        if mode.lower() == 'd':
            block.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, padding_mode='reflect'))

        elif mode.lower() == 'u':
            block.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, output_padding=1)) # output size를 input이랑 같게 해주려면 이렇게 설정을 해줄 수밖에 없음.

        block += [nn.InstanceNorm2d(out_channels), nn.ReLU(inplace=True)]

        return nn.Sequential(*block)


#Discriminator

shape_dict=dict() # for checking the output's shape

class Discriminator(nn.Module):
    def __init__(self, n_layers:int=4, input_c:int=3, n_filter:int=64, kernel_size:int=4):

        super(Discriminator, self).__init__()
        self.model = nn.Sequential()
        self.kernel_size=kernel_size
        self.n_layers = n_layers
        layers = []

        # building conv block
        for i in range(self.n_layers):
            if i == 0:
                ic, oc = input_c, n_filter
                layers.append(self._make_block(ic, oc, kernel_size=self.kernel_size, stride=2, padding=1, normalize=False))
            else:
                ic = oc
                oc = 2*ic
                stride=2

                if i == self.n_layers-1: # 마지막 레이어(c512)의 경우, stride=1로 설정할 것.
                    stride=1

                layers.append(self._make_block(ic, oc, kernel_size=self.kernel_size, stride=stride, padding=1))

        # prediction
        layers.append(nn.Conv2d(oc, 1, kernel_size=self.kernel_size, stride=1, padding=1))

        self.model = nn.Sequential(*layers)


    def forward(self, x):
        return self.model(x)


    def _make_block(self, in_channels, out_channels, stride, kernel_size=3, padding=0, normalize=True):
        layers = [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=stride, kernel_size=kernel_size, padding=padding)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2, inplace=True))

        return nn.Sequential(*layers)

# check the output size within a model

# hook function은 forward 이후에 activate된다는 점을 잊지 말자.
def hook_fn(m, _, o):
    shape_dict[m]=o.shape


def get_all_layers(net:nn.Module, hook_fn=hook_fn):
    for name, layer in net._modules.items():
        #print(name)
        if isinstance(layer, nn.Sequential):
            get_all_layers(layer)
        else:
            layer.register_forward_hook(hook_fn)

In [21]:
from torch.optim.lr_scheduler import ConstantLR, _LRScheduler, SequentialLR
from torch.optim.optimizer import Optimizer
import warnings

def decay_after_k_epoch(optim: Optimizer, n_epochs:int, init_lr: float, target_lr: float, decay_after:int=100):
    scheduler1 = ConstantLR(optim, factor=1, total_iters=decay_after)
    scheduler2 = LinearDecayLR(optim, initial_lr = init_lr, target_lr=target_lr, total_iters=n_epochs-decay_after) # constant는 그냥 한 번 줄여주고 마는 거임.

    scheduler = SequentialLR(optim, schedulers=[scheduler1, scheduler2], milestones=[decay_after])
    return scheduler


class LinearDecayLR(_LRScheduler):
    def __init__(self, optimizer:Optimizer, initial_lr: float, target_lr:float, total_iters:int, last_epoch:int=-1, verbose:bool=False):

        if initial_lr < 0:
            raise ValueError("Initial Learning rate expected to be a non-negative integer.")

        if target_lr < 0:
            raise ValueError("Target Learning rate expected to be a non-negative integer.")

        if target_lr > initial_lr:
            raise ValueError("Target Learning Rate must be larger than Initial Learning Rate.")

        self.init_lr = initial_lr
        self.target_lr = target_lr
        self.total_iters = total_iters
        self.subtract_lr = self._get_decay_constant()

        super(LinearDecayLR, self).__init__(optimizer, last_epoch, verbose) # 부모클래스의 init에 필요한 arg 넘겨줌.
        # super init을 해도 self.base_lrs를 왜 상속을 못받는 거지?
        # 여기서 부모 메소드 init 과정에서 self.get_lr을 호출함. 근데 get_lr에는 self.subtract_lr이 들어가 있고, 그건 다시 부모클래스 init이 "완료"되어야 하기 때문에
        # 아래 명령을 실행한 적이 없음. 그래서 계속 그런 attribute이 없다고 뜨는 거임.


    def _get_decay_constant(self):
        return float((self.init_lr-self.target_lr)/self.total_iters)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the learning rate computed by the scheduler, "
                        "please use 'get_last_lr()'.", UserWarning)

        return [group['lr']-self.subtract_lr for group in self.optimizer.param_groups]


class DelayedLinearDecayLR(LinearDecayLR):
    def __init__(self, optimizer:Optimizer, initial_lr: float, target_lr: float, total_iters:int, last_epoch:int=-1, decay_after:int=100, verbose:bool=False):
        self.decay_after = decay_after

        super(DelayedLinearDecayLR, self).__init__(optimizer, initial_lr, target_lr, total_iters, last_epoch, verbose)


    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the learning rate computed by the scheduler, "
                        "please use 'get_last_lr()'.", UserWarning)

        if self.decay_after <= self.last_epoch < (self.decay_after + self.total_iters): # 여기에 total iter도 고려해줘야함.
            return [group['lr']-self.subtract_lr for group in self.optimizer.param_groups]

        else:
            return [group['lr'] for group in self.optimizer.param_groups]

In [22]:
import argparse
import cv2
import numpy as np
import os
import random
import torch
from torch.optim.optimizer import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from typing import Optional
from torchvision.utils import make_grid
import torchvision.transforms.functional as TF
import math

#from augmentation_my import denormalize_image

def fix_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

def parse_opt():
    opt=argparse.Namespace(
        is_train=True,
        num_threads=1,
        gpu_id=0,
        random_seed=0,
        data_root_A="/content/drive/MyDrive/day_1",
        data_root_B="/content/drive/MyDrive/night_1",
        batch_size=8,
        n_epochs=4,
        beta1=.5,
        lr=.0002,
        decay_after=100,
        target_lr=0.0,
        total_iters=100,
        lr_decay_verbose=False,
        lambda_cyc=10.0,
        lambda_idt=0.5,
        prj_name="cycle_gan",
        exp_name="exp1",
        log_interval=25,
        sample_save_dir="/content/drive/MyDrive/result", #각 epoch.jpg가 저장되는 곳
        checkpoint_dir="/content/drive/MyDrive/save_folder", #epoch.pth가 저장되는 곳
        load_epoch=0, #n_epochs 즉 맨 마지막 epoch의 값이 기본값이고, 얼리스탑 기능이 작동하면 값이 변하고 그 값의 폴더를 사용해서 테스트한다.
        last_epoch=0, # 다시 시작할때 0값을 마지막 폴더의 숫자로 바꾸고 시작
        resume_from = False, # True는 불러올 데이터가 있을때, 즉 이걸 돌리다가 불가피하게 끊겼을때 다시 돌릴 수 있게 하는 것. 그러므로 처음 할 때는 False로 해야함
        data_root_A_final="/content/drive/MyDrive/cameraday", #테스트 낮 폴더
        data_root_B_final="/content/drive/MyDrive/cameranight" #테스트 밤 폴더





        )
    return opt

def save_image(image:torch.Tensor, save_path:str, denormalize:bool=True):
    """
    convert a torch.Tensor to a numpy array-> denormalize (if needed) -> type casting -> writing image to the given path
    """
    # if isinstance(image, torch.Tensor):
    #     image = image.numpy()

    if denormalize:
        image = denormalize_image(image)

    #image = image.astype(np.uint8).copy()
    image=make_grid(image.cpu().detach(),nrow=int(math.sqrt(image.size(0))))
    image=TF.to_pil_image(image)
    image.save(save_path)

   # cv2.imwrite(save_path, cv2.cvtColor(image.transpose(1,2,0), cv2.COLOR_BGR2RGB))


def save_checkpoint(epoch, G, F, D_x, D_y, optim_G, optim_D, scheduler_G, scheduler_D, saved_dir, file_name):
    check_point = {'epoch': epoch,
                    'G': G.state_dict(),
                    'F': F.state_dict(),
                    'D_x': D_x.state_dict(),
                    'D_y': D_y.state_dict(),
                    'optimG_state_dict': optim_G.state_dict(),
                    'optimD_state_dict': optim_D.state_dict()
                    }
    if scheduler_G and scheduler_D:
        check_point['scheduler_G_state_dict'] = scheduler_G.state_dict()
        check_point['scheduler_D_state_dict'] = scheduler_D.state_dict()

    os.makedirs(saved_dir, exist_ok=True) # make a directory to save a model if not exist.

    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point, output_path)


def load_checkpoint(checkpoint_path, G, F, D_x:Optional[torch.nn.Module]=None, D_y:Optional[torch.nn.Module]=None, \
                    optim_G:Optional[Optimizer]=None, optim_D:Optional[Optimizer]=None, \
                    scheduler_G:Optional[_LRScheduler]=None, scheduler_D:Optional[_LRScheduler]=None, mode:str="model"):
    # load model if resume_from is set
    checkpoint = torch.load(checkpoint_path)
    G.load_state_dict(checkpoint['G'])
    F.load_state_dict(checkpoint['F'])

    if D_x and D_y:
        D_x.load_state_dict(checkpoint['D_x'])
        D_y.load_state_dict(checkpoint['D_y'])

    start_epoch = checkpoint['epoch']

    if mode == "model":
        return G, F, D_x, D_y, start_epoch

    if mode =="all":
        optim_G.load_state_dict(checkpoint['optimG_state_dict'])
        optim_D.load_state_dict(checkpoint['optimD_state_dict'])

        if scheduler_G and scheduler_D:
            scheduler_G.load_state_dict(checkpoint['scheduler_G_state_dict'])
            scheduler_D.load_state_dict(checkpoint['scheduler_D_state_dict'])

            return G, F, D_x, D_y, optim_G, optim_D, scheduler_G, scheduler_D, start_epoch

        return G, F, D_x, D_y, optim_G, optim_D, start_epoch

    else:
        raise ValueError("mode should be one of 'model' or 'all'")

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


from sched import scheduler
import torch
from torch.utils.data import DataLoader

# from augmentation_my import BaseAugmentation
# from criterion_my import *
# from dataset_my import UnalignedDataset
# from model_my import *
# from utils_my import *
# from scheduler_my import DelayedLinearDecayLR

from functools import partial
import itertools
#import os
from tqdm import tqdm
# import wandb
import argparse

# opt로 변경하는 작업
def train(opt):

    os.makedirs(opt.sample_save_dir, exist_ok=True)

    # device
    device = torch.device(f'cuda:{opt.gpu_id}' if torch.cuda.is_available() else 'cpu')

    # define transforms
    transforms = BaseAugmentation()

    # load datasets
    train_data = UnalignedDataset(opt.data_root_A, opt.data_root_B, opt.is_train, transforms = transforms.transform)
    train_loader = DataLoader(train_data, batch_size=opt.batch_size, num_workers=opt.num_threads, shuffle=True)

    # load model
    G = Generator(init_channel=64, kernel_size=3, stride=2, n_blocks=9).to(device)
    F = Generator(init_channel=64, kernel_size=3, stride=2, n_blocks=9).to(device)
    D_x = Discriminator().to(device)
    D_y = Discriminator().to(device)

    # define optimizer
    optimizer = partial(torch.optim.Adam, lr=opt.lr, betas=(opt.beta1, 0.999))

    optim_G = optimizer(params = itertools.chain(G.parameters(), F.parameters()))
    optim_D = optimizer(params = itertools.chain(D_x.parameters(), D_y.parameters()))

    # scheduler
    scheduler_G = DelayedLinearDecayLR(optim_G, opt.lr, opt.target_lr, opt.total_iters, decay_after=opt.decay_after, verbose=opt.lr_decay_verbose)
    scheduler_D = DelayedLinearDecayLR(optim_D, opt.lr, opt.target_lr, opt.total_iters, decay_after=opt.decay_after, verbose=opt.lr_decay_verbose)


    criterion_G = AdversarialLoss()
    criterion_D = AdversarialLoss()
    criterion_cyc = CycleConsistencyLoss()
    criterion_idt = IdentityLoss()

    G.train()
    F.train()
    D_x.train()
    D_y.train()

    if opt.resume_from:
        checkpoint_path = os.path.join(opt.checkpoint_dir, f"epoch{opt.last_epoch}.pth")
        if "LinearDecay" in getattr(type(scheduler_D), '__name__'): # when using LinearDecayLR, not loading state dict for optimizers and schedulers.
            G, F, D_x, D_y, start_epoch = load_checkpoint(checkpoint_path, G, F, D_x, D_y)
            # decay after 같은 부분 수정하는 건 그냥 config 자체에서 받아오는 걸로 / last epoch만 수정해주자.
            scheduler_dict = {
                'decay_after': start_epoch + opt.decay_after,
                'last_epoch': opt.last_epoch
            }
            scheduler_G.load_state_dict(scheduler_dict)
            scheduler_D.load_state_dict(scheduler_dict)

        else:
            G, F, D_x, D_y, optim_G, optim_D, scheduler_G, scheduler_D, start_epoch = load_checkpoint(checkpoint_path, G, F, D_x, D_y, \
                                                                                                    optim_G, optim_D, scheduler_G, scheduler_D, mode="all")
        start_epoch += 1
    else: # not_resume from
        start_epoch = 0

    for epoch in range(start_epoch, start_epoch+opt.n_epochs):
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for step, data in pbar:

            # Initialize the gradient stored in the optimizer to zero in the beginning of each step.
            optim_G.zero_grad()
            optim_D.zero_grad()

            # load data on gpu
            X, Y = data['A'].to(device), data['B'].to(device)

            #### Generator ####
            for p_x, p_y in zip(D_x.parameters(), D_y.parameters()): # when optimizing G, freeze the parameters regarding D.
                p_x.requires_grad = False
                p_y.requires_grad = False

            # generation & reconstruction
            g_x = G(X) # fake_B
            f_y = F(Y) # fake_A
            rec_x = F(g_x) # rec_A (reconstruction)
            rec_y = G(f_y) # rec_B

            # discriminating the generators' outputs.
            d_g_x = D_y(g_x)
            d_f_y = D_x(f_y)

            # generate the label
            real_label = torch.tensor([1.0]).expand_as(d_g_x).to(device)
            fake_label = torch.tensor([0.0]).expand_as(d_f_y).to(device)

            # calculate an adversarial loss -> maximize the probability of the fake to be "considered" real
            loss_G_xy = criterion_G.forward_G(d_g_x, real_label)
            loss_F_yx = criterion_G.forward_G(d_f_y, real_label)

            # calc cycle loss
            loss_cyc = criterion_cyc(X, Y, rec_x, rec_y)

            # calc identity loss if lambda of identity loss larger than zero.
            if opt.lambda_idt > 0:
                loss_idt = opt.lambda_idt*criterion_idt(X, Y, f_y, g_x)
            else: # lambda_idt = 0
                loss_idt = 0

            loss_G = loss_G_xy + loss_F_yx + opt.lambda_cyc*loss_cyc + opt.lambda_idt*loss_idt

            loss_G.backward() # calculate all the gradient with respect to loss_G.
            optim_G.step() # alternating training 해야돼서 G랑 D는 optimizer 따로 쓰는 거임.

            #### Discriminator ####
            for p_x, p_y in zip(D_x.parameters(), D_y.parameters()):
                p_x.requires_grad = True
                p_y.requires_grad = True

            loss_D_xy = criterion_D.forward_D(D_y(Y), real_label, D_y(g_x.detach()), fake_label)
            loss_D_yx = criterion_D.forward_D(D_x(X), real_label, D_x(f_y.detach()), fake_label)

            # average the loss
            loss_D = (loss_D_xy+loss_D_yx)/2

            loss_D.backward()
            optim_D.step()


        # Apply the scheduler (make sure the step of optimizers precede that of schedulers)
        scheduler_G.step()
        scheduler_D.step()

        # saving checkpoints
        checkpoint_dir = os.path.join(opt.checkpoint_dir, opt.exp_name)
        opt.load_epoch=epoch


        best_loss=float("inf")
        if loss_cyc.item()<best_loss:
            best_loss=loss_cyc.item()
            early_stop_counter=0
        else:
            early_stop_counter+=1
        if early_stop_counter>=5:
            print(f"Early stopping at epoch, which is {epoch}")
            break

        #if epoch%5==0:
        save_checkpoint(epoch, G, F, D_x, D_y, optim_G, optim_D, scheduler_G, scheduler_D, checkpoint_dir, file_name=f"epoch{epoch+1}.pth")

        # saving sample outputs
        if opt.log_interval>0:
             save_image(g_x.clone().detach().cpu(), f"{opt.sample_save_dir}/epoch{epoch+1}_step{step+1}.jpg")




def main():
    opt = parse_opt()

    fix_seed(opt.random_seed) # randomness 제어/
    train(opt)


if __name__ == "__main__":
    main()

 49%|████▉     | 56/114 [02:48<02:53,  3.00s/it]

In [23]:
import os
from tqdm import tqdm

# from augmentation_my import BaseAugmentation
# from dataset_my import UnalignedDataset
# from model_my import Generator
# from utils_my import *

import torch
#from torch.utils.data import DataLoader

def test(test_loader, G, F, device, save_dir,save_dir_XtoY='/content/drive/MyDrive/result/exp1/epoch_final/day2night', save_dir_YtoX='/content/drive/MyDrive/result/exp1/epoch_final/night2day'):
    G.to(device)
    F.to(device)

    G.eval()
    F.eval()

    with torch.no_grad():
        pbar = tqdm(enumerate(test_loader), total=len(test_loader))

        for step, data in pbar:
            X, Y = data['A'], data['B']

            X, Y = X.to(device), Y.to(device)

            fake_Y = G(X)
            fake_X = F(Y)

            # make an image array
            XY = torch.cat([X, fake_Y, F(fake_Y)], dim = 3) # column-wise concat
            YX = torch.cat([Y, fake_X, G(fake_X)], dim = 3)

            os.makedirs(f"{save_dir_XtoY}/", exist_ok=True)
            os.makedirs(f"{save_dir_YtoX}/", exist_ok=True)

            # array로 저장해서 보는 게 빠를 것 같음. grid로 만들어서 보는 법 찾아볼 것.
            save_image(XY.clone().detach().cpu(), f"{save_dir}/day2night/{step+1}.png")
            save_image(YX.clone().detach().cpu(), f"{save_dir}/night2day/{step+1}.png")




if __name__ == "__main__":
    opt = parse_opt()

    fix_seed(opt.random_seed) # randomness 제어

    # device
    device = torch.device(f'cuda:{opt.gpu_id}' if torch.cuda.is_available() else 'cpu')

    # define transforms
    transforms = BaseAugmentation()

    # load datasets
    test_data = UnalignedDataset(opt.data_root_A_final, opt.data_root_B_final, False, transforms = transforms.transform)
    test_loader = DataLoader(test_data, batch_size=opt.batch_size, num_workers=opt.num_threads, shuffle=False)

    # declare architectures
    G = Generator(init_channel=64, kernel_size=3, stride=2, n_blocks=9)
    F = Generator(init_channel=64, kernel_size=3, stride=2, n_blocks=9)

    G, F, _, _, _ = load_checkpoint(os.path.join(opt.checkpoint_dir,opt.exp_name,f"epoch{opt.load_epoch+1}.pth"), G, F)

    save_dir = os.path.join(opt.sample_save_dir, opt.exp_name, f"epoch_final")
    os.makedirs(save_dir, exist_ok=True)

    test(test_loader, G, F, device, save_dir)

100%|██████████| 1/1 [00:01<00:00,  1.56s/it]
